In [0]:
import torch
import logging
import time
import torch.optim as optim

In [0]:
def pretrain(trainloader, autoencoder, device='cuda', lr=1e-3, 
             milestones=[20, 40, 60], ae_epochs=100, accumulation_steps=64,
             weight_decay=1e-6, accumulate=False):
    
    logger = logging.getLogger()

    # Set device for network
    autoencoder = autoencoder.to(device)

    # Set optimizer (Adam optimizer for now)
    optimizer = optim.Adam(autoencoder.parameters(), lr=lr, 
                           weight_decay=weight_decay, amsgrad=True)

    # Set learning rate scheduler
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=0.1)
    
    if not accumulate:
        accumulation_steps = 0
    # Training
    logger.info('Starting pretraining...')
    logger.info('Learning rate : {}'.format(lr))
    logger.info('Gradient accumulation : {}'.format(accumulate))
    logger.info('Accumulation steps : {}'.format(accumulation_steps))
    logger.info('AE epochs : {}'.format(ae_epochs))
    
    start_time = time.time()
    autoencoder.train()

    for epoch in range(ae_epochs):
        scheduler.step()
        if epoch in milestones:
            logger.info('  LR scheduler: new learning rate is %g' % float(scheduler.get_lr()[0]))

        loss_epoch = 0.0
        n_batches = 0
        epoch_start_time = time.time()
        for i, data in enumerate(trainloader):
            inputs, _, _ = data
            inputs = inputs.to(device)

            # Zero the network parameter gradients
            optimizer.zero_grad()

            # Update network parameters via backpropagation: forward + backward + optimize
            outputs = autoencoder(inputs)
            scores = torch.sum((outputs - inputs) ** 2, dim=tuple(range(1, outputs.dim())))
            loss = torch.mean(scores)
            if accumulate:
                loss = loss / accumulation_steps
                loss.backward()
                if (i+1) % accumulation_steps == 0:             # Wait for several backward steps
                    optimizer.step()                            # Now we can do an optimizer step
                    autoencoder.zero_grad()                           # Reset gradients tensors
            else:
                loss.backward()
                optimizer.step()

            loss_epoch += loss.item()
            n_batches += 1

        # log epoch statistics
        epoch_train_time = time.time() - epoch_start_time
        logger.info('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f}'
                    .format(epoch + 1, ae_epochs, epoch_train_time, loss_epoch / n_batches))

    pretrain_time = time.time() - start_time
    logger.info('Pretraining time: %.3f' % pretrain_time)
    logger.info('Finished pretraining.')

    return autoencoder